In [3]:
%pylab notebook

Populating the interactive namespace from numpy and matplotlib


In [4]:
from Fisher_CMB import *
from Models.SPT3G.spt3G_model import spt3G_model
from Models.Planck.Planck_model import planck_model
from pickle import load as pload
from pickle import dump
from Covariances.create_basic_cov import *
import pandas as pd
import cosmoslik as K
import matplotlib.lines as mlines

# Make Fisher Matrices and Save 

##  LCDM Fisher Matrices

In [28]:
params={'H0':fish_param(67.90,0.55),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           'lmax':6000,'lens_potential_accuracy':5.0}

In [16]:
params3g={'H0':fish_param(67.90,0.55),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                            'Asz':fish_param(5.5,3,3,nuisance=True),
                            'Aps':fish_param(19.3,3.5,3.5,nuisance=True),
                            'Acib':fish_param(5,2.5,2.5,nuisance=True),
                            'A_TEps' : fish_param(.16,.05,nuisance=True),
                            'Pcal':fish_param(1.06,.01,.01,nuisance=True),
                           'lmax':6000,'lens_potential_accuracy':5.0}

### Planck TT $\ell$ < 800 Fisher matrix

In [30]:
pcov_800=create_basic_cov(.7,[.0003],[0,'ukarc'],
                     specrange = (2,800),
                     bin_size = 5,
                     spectra=['TT'],
                     )

full_pcov_800 = make_full_cov(['TT'],pcov_800)


Fp_800,names=Fisher_Matrix(params,full_pcov_800,
                 planck_model(specrange = [('TT',(2,800))],bin_size=5),show_results=False)

pd.DataFrame(Fp_800,index=names,columns=names).to_hdf('Saved_Fisher_Matrices/planck_lcdm_800_Fisher_Matrix.h5','key')

/nfs/home/kmaylor/Forecasting/Covariances/create_basic_cov.py:37: RuntimeWarning: divide by zero encountered in true_divide
  con=lambda n:arcmin_per_sr/n**2


### Planck $\ell$ < 2500 Fisher Matrix

In [20]:
def add_params(planck):
    planck['ommh2']=planck['omegabh2']+planck['omegach2']
    planck['H0']=planck['H0*']
    planck['clamp']=planck['clamp*']
    planck['ombh2']=planck['omegabh2']
    planck['YHe']=planck.get('yhe',planck['H0*'])
    planck['omk']=planck.get('omegak',planck['H0*'])
    return planck

In [47]:
Planck_path="/nfs/home/kmaylor/Official_Planck_chains/base/plikHM_TTTEEE_lowTEB/base_plikHM_TTTEEE_lowTEB" #loading planck 2015 chain
Planck_chain=K.chains.load_cosmomc_chain(Planck_path).burnin(1000).join()
Planck_chain=add_params(Planck_chain)

cov_params=sorted(['H0','ombh2','ommh2','tau','clamp','ns'])
Planck_cov = Planck_chain.cov(cov_params)
Fp=inv(Planck_cov)

pd.DataFrame(Fp,index=cov_params,columns=cov_params).to_hdf('Saved_Fisher_Matrices/planck_lcdm_Fisher_Matrix.h5','key')

### SPT-SZ Fisher Matrix

In [48]:
def slik2cmc( c ):
    c['ns'] = c['cosmo.ns']
    c['ombh2'] = c['cosmo.ombh2']
    c['omch2'] = c['cosmo.omch2']
    c['tau'] = c['cosmo.tau']
    c['clamp'] = exp(c['cosmo.logA'])*exp(-2*c['tau'])/10
    c['ommh2']=c['ombh2']+c['omch2']
    c['H0'] = c['cosmo.H0']
    return c

In [49]:
SPTSZ_path="/home/kmaylor/Python_Projects/cosmology_chains/SPT_chainz_150x150.chains"
SPTSZ_chain=K.load_chain(SPTSZ_path).burnin(500).join()
SPTSZ_chain = slik2cmc(SPTSZ_chain)
cov_params=sorted(['H0','ombh2','ommh2','tau','clamp','ns'])
SPTSZ_cov = SPTSZ_chain.cov(cov_params)
Fsz=inv(SPTSZ_cov)
pd.DataFrame(Fsz,index=cov_params,columns=cov_params).to_hdf('Saved_Fisher_Matrices/SPTSZ_lcdm_Fisher_Matrix.h5','key')

### SPT3G Fisher Matrix

In [10]:
def SPT3G_Fisher_for_lmin(fsky,lmin_range,beam_width,noise,P_lmax,T_lmax,spectra,binn,filename):
    lmin_dic={}

    for lmin in arange(lmin_range[0],lmin_range[1],100):
        spt3g_cov=create_basic_cov(fsky,beam_width,noise,specrange = (lmin,P_lmax),bin_size = binn,spectra=spectra)
    
        full_spt3g_cov=make_full_cov(spectra,spt3g_cov)

        Fs,names=Fisher_Matrix(params3g,full_spt3g_cov,
                 spt3G_model(specrange = list(zip(spectra,[(lmin,T_lmax),(lmin,P_lmax),(lmin,P_lmax)])),
                             bin_size=binn),show_results=False)
    
        lmin_dic[str(lmin)]=pd.DataFrame(Fs,index=names,columns=names)
    lmin_pn=pd.Panel(lmin_dic)
    lmin_pn.to_hdf(filename,'key')

In [6]:
SPT3G_Fisher_for_lmin(fsky=0.06,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([3.6,3.3,8.5]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_lcdm_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


### SPT3G Fisher Matrix fsky =0.035

In [7]:
SPT3G_Fisher_for_lmin(fsky=0.035,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([2.8,2.6,6.6]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_f035_lcdm_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


###  SPT3G lensing reconstruction Fisher Matrix

In [3]:
paramsphi={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           #'mnu':fish_param(.06,0.01),
                           #'nnu':fish_param(3.046,0.03),
                           #'YHe':fish_param(.252,.014),
                            #'Asz':fish_param(5.5,3,3,nuisance=True),
                            #'Aps':fish_param(19.3,3.5,3.5,nuisance=True),
                            #'Acib':fish_param(5,2.5,2.5,nuisance=True),
                            #'A_TEps' : fish_param(.16,.05,nuisance=True),
                            #'Pcal':fish_param(1.06,.01,.01,nuisance=True),
                           'lmax':6000,'lens_potential_accuracy':5.0}

In [11]:
def SPT3G_phi_Fisher_for_lmin(fsky,lmin_range,lmax,binn,filename):
    lmin_dic={}

    for lmin in arange(lmin_range[0],lmin_range[1],100):
        m=spt3G_model(specrange = [('PP',(lmin,lmax))],bin_size=binn)

        lensing_recon_cov_file = 'Covariances/spt3g_3.3uk_lmaxTT3000_lmaxPOL5000_nx1024_plikHM_TT_lowTEB_lensing.dat'

        PP_cov=diag(dot(m.windows['PP'],
                (loadtxt(lensing_recon_cov_file)[:3000,-1]*(2.725*1e6)**2)[m.windowrange['PP']]))
        
        Fs_phi,names=Fisher_Matrix(paramsphi,PP_cov,
                                   spt3G_model(specrange = [('PP',(lmin,lmax))],bin_size=binn),show_results=False)
        lmin_dic[str(lmin)]=pd.DataFrame(Fs_phi,index=names,columns=names)
    lmin_pn=pd.Panel(lmin_dic)
    lmin_pn.to_hdf(filename,'key')

In [8]:
SPT3G_phi_Fisher_for_lmin(0.06,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_lcdm_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  """Entry point for launching an IPython kernel.


In [9]:
SPT3G_phi_Fisher_for_lmin(0.035,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  """Entry point for launching an IPython kernel.


## LCDM + Neff

In [8]:
params={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           #'mnu':fish_param(.06,0.01),
                           'nnu':fish_param(3.046,0.03),
                           #'YHe':fish_param(.252,.014),
                           'lmax':6000,'lens_potential_accuracy':5.0}

In [8]:
params3g={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           #'mnu':fish_param(.06,0.01),
                           'nnu':fish_param(3.046,0.03),
                           #'YHe':fish_param(.252,.014),
                            'Asz':fish_param(5.5,3,3,nuisance=True),
                            'Aps':fish_param(19.3,3.5,3.5,nuisance=True),
                            'Acib':fish_param(5,2.5,2.5,nuisance=True),
                            'A_TEps' : fish_param(.16,.05,nuisance=True),
                            'Pcal':fish_param(1.06,.01,.01,nuisance=True),
                           'lmax':6000,'lens_potential_accuracy':5.0}

### Planck $\ell$ < 800

In [131]:
Fp_800_neff,names=Fisher_Matrix(params,full_pcov_800,
                 planck_model(specrange = [('TT',(2,800))],bin_size=5),show_results=False)

pd.DataFrame(Fp_800_neff,index=names,columns=names).to_hdf('Saved_Fisher_Matrices/planck_lcdm_neff_800_Fisher_Matrix.h5','key')

### Planck $\ell$ < 2500 Fisher Matrix

In [100]:
Planck_neff_path="/nfs/home/kmaylor/Official_Planck_chains/base_nnu/plikHM_TTTEEE_lowTEB/base_nnu_plikHM_TTTEEE_lowTEB" #loading planck 2015 chain
Planck_neff_chain=K.chains.load_cosmomc_chain(Planck_neff_path).burnin(1000).join()
Planck_neff_chain=add_params(Planck_neff_chain)

cov_params_neff=sorted(['H0','ombh2','ommh2','tau','clamp','ns','nnu'])
Planck_neff_cov = Planck_neff_chain.cov(cov_params_neff)
Fp_neff=inv(Planck_neff_cov)

pd.DataFrame(Fp_neff,index=cov_params_neff,columns=cov_params_neff).to_hdf('Saved_Fisher_Matrices/planck_lcdm_neff_Fisher_Matrix.h5','key')

### SPT3G Fisher Matrix

In [9]:
SPT3G_Fisher_for_lmin(fsky=0.06,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([3.6,3.3,8.5]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_lcdm_neff_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


### SPT3G Fisher Matrix fsky =0.035

In [10]:
SPT3G_Fisher_for_lmin(fsky=0.035,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([2.8,2.6,6.6]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_f035_lcdm_neff_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


###  SPT3G lensing reconstruction Fisher Matrix

In [10]:
paramsphi={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           #'mnu':fish_param(.06,0.01),
                           'nnu':fish_param(3.046,0.03),
                           'lmax':6000,'lens_potential_accuracy':5.0}

SPT3G_phi_Fisher_for_lmin(0.06,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_lcdm_neff_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  # This is added back by InteractiveShellApp.init_path()


In [11]:
SPT3G_phi_Fisher_for_lmin(0.035,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_lmin_neff_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  """Entry point for launching an IPython kernel.


##  LCDM + Neff + Yp

In [ ]:
params={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           #'mnu':fish_param(.06,0.01),
                           'nnu':fish_param(3.046,0.03),
                           'YHe':fish_param(.252,.014),
                           'lmax':6000,'lens_potential_accuracy':5.0}

In [11]:
params3g={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           #'mnu':fish_param(.06,0.01),
                           'nnu':fish_param(3.046,0.03),
                           'YHe':fish_param(.252,.014),
                           'Asz':fish_param(5.5,3,3,nuisance=True),
                            'Aps':fish_param(19.3,3.5,3.5,nuisance=True),
                            'Acib':fish_param(5,2.5,2.5,nuisance=True),
                            'A_TEps' : fish_param(.16,.05,nuisance=True),
                            'Pcal':fish_param(1.06,.01,.01,nuisance=True),
                           'lmax':6000,'lens_potential_accuracy':5.0}

### Planck $\ell$ < 800


In [136]:
Fp_800_neff_yp,names=Fisher_Matrix(params,full_pcov_800,
                 planck_model(specrange = [('TT',(2,800))],bin_size=5),show_results=False)

pd.DataFrame(Fp_800_neff_yp,index=names,columns=names).to_hdf('Saved_Fisher_Matrices/planck_lcdm_neff_yp_800_Fisher_Matrix.h5','key')

### Planck $\ell$ < 2500 Fisher Matrix

In [101]:
Planck_neff_yp_path="/nfs/home/kmaylor/Official_Planck_chains/base_nnu_yhe/plikHM_TTTEEE_lowTEB/base_nnu_yhe_plikHM_TTTEEE_lowTEB" #loading planck 2015 chain
Planck_neff_yp_chain=K.chains.load_cosmomc_chain(Planck_neff_yp_path).burnin(1000).join()
Planck_neff_yp_chain=add_params(Planck_neff_yp_chain)

cov_params_neff_yp=sorted(['H0','ombh2','ommh2','tau','clamp','ns','nnu','YHe'])
Planck_neff_yp_cov = Planck_neff_yp_chain.cov(cov_params_neff_yp)
Fp_neff_yp=inv(Planck_neff_yp_cov)

pd.DataFrame(Fp_neff_yp,index=cov_params_neff_yp,columns=cov_params_neff_yp).to_hdf('Saved_Fisher_Matrices/planck_lcdm_neff_yp_Fisher_Matrix.h5','key')

### SPT3G Fisher Matrix

In [12]:
SPT3G_Fisher_for_lmin(fsky=0.06,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([3.6,3.3,8.5]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_lcdm_neff_yp_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


### SPT3G Fisher Matrix fsky =0.035

In [13]:
SPT3G_Fisher_for_lmin(fsky=0.035,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([2.8,2.6,6.6]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_f035_lcdm_neff_yp_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


###  SPT3G lensing reconstruction Fisher Matrix

In [12]:
paramsphi={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           #'mnu':fish_param(.06,0.01),
                           'nnu':fish_param(3.046,0.03),
                           'YHe':fish_param(.252,.014),
                           'lmax':6000,'lens_potential_accuracy':5.0}
SPT3G_phi_Fisher_for_lmin(0.06,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_lcdm_neff_yp_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  # This is added back by InteractiveShellApp.init_path()


In [13]:
SPT3G_phi_Fisher_for_lmin(0.035,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_neff_yp_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  """Entry point for launching an IPython kernel.


##  LCDM + $\Sigma m_{\nu}$

In [14]:
params={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           'mnu':fish_param(.06,0.01),
                           #'nnu':fish_param(3.046,0.03),
                           #'YHe':fish_param(.252,.014),
                           'lmax':6000,'lens_potential_accuracy':5.0}

In [15]:
params3g={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           'mnu':fish_param(.06,0.01),
                           #'nnu':fish_param(3.046,0.03),
                           #'YHe':fish_param(.252,.014),
                           'Asz':fish_param(5.5,3,3,nuisance=True),
                            'Aps':fish_param(19.3,3.5,3.5,nuisance=True),
                            'Acib':fish_param(5,2.5,2.5,nuisance=True),
                            'A_TEps' : fish_param(.16,.05,nuisance=True),
                            'Pcal':fish_param(1.06,.01,.01,nuisance=True),
                           'lmax':6000,'lens_potential_accuracy':5.0}

### Planck $\ell$ < 800


In [18]:
pcov_800=create_basic_cov(.7,[.0003],[0,'ukarc'],
                     specrange = (2,800),
                     bin_size = 5,
                     spectra=['TT'],
                     )

full_pcov_800 = make_full_cov(['TT'],pcov_800)

Fp_800_mnu,names=Fisher_Matrix(params,full_pcov_800,
                 planck_model(specrange = [('TT',(2,800))],bin_size=5),show_results=False)

pd.DataFrame(Fp_800_mnu,index=names,columns=names).to_hdf('Saved_Fisher_Matrices/planck_lcdm_mnu_800_Fisher_Matrix.h5','key')

/nfs/home/kmaylor/Forecasting/Covariances/create_basic_cov.py:37: RuntimeWarning: divide by zero encountered in true_divide
  con=lambda n:arcmin_per_sr/n**2


### Planck $\ell$ < 2500 Fisher Matrix

In [ ]:
Planck_mnu_path="/nfs/home/kmaylor/Official_Planck_chains/base_mnu/plikHM_TTTEEE_lowTEB_lensing/base_mnu_plikHM_TTTEEE_lowTEB_lensing" #loading planck 2015 chain
Planck_mnu_chain=K.chains.load_cosmomc_chain(Planck_mnu_path).burnin(1000).join()
Planck_mnu_chain=add_params(Planck_mnu_chain)

cov_params_mnu=sorted(['H0','ombh2','ommh2','tau','clamp','ns','mnu'])
Planck_mnu_cov = Planck_mnu_chain.cov(cov_params_mnu)
Fp_mnu=inv(Planck_mnu_cov)

pd.DataFrame(Fp_mnu,index=cov_params_mnu,columns=cov_params_mnu).to_hdf('Saved_Fisher_Matrices/planck_lcdm_mnu_Fisher_Matrix.h5','key')

### SPT3G Fisher Matrix

In [24]:
SPT3G_Fisher_for_lmin(fsky=0.06,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([3.6,3.3,8.5]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_lcdm_mnu_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


### SPT3G Fisher Matrix fsky =0.035

In [25]:
SPT3G_Fisher_for_lmin(fsky=0.035,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([2.8,2.6,6.6]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_f035_lcdm_mnu_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  if __name__ == '__main__':


###  SPT3G lensing reconstruction Fisher Matrix

In [14]:
paramsphi={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           'mnu':fish_param(.06,0.01),
                           #'nnu':fish_param(3.046,0.03),
                           #'YHe':fish_param(.252,.014),
                           'lmax':6000,'lens_potential_accuracy':5.0}
SPT3G_phi_Fisher_for_lmin(0.06,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_lcdm_mnu_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  # This is added back by InteractiveShellApp.init_path()


In [15]:
SPT3G_phi_Fisher_for_lmin(0.035,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_mnu_lmin_100_700_Fisher_Matrices.h5')

/nfs/home/kmaylor/.virtualenvs/Forecasting_env/local/lib/python3.4/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  """Entry point for launching an IPython kernel.


##  LCDM + $\Omega K$

In [5]:
params={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           'omk':fish_param(-0.003687,0.007),
                           'lmax':6000,'lens_potential_accuracy':5.0}

In [6]:
params3g={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           'omk':fish_param(-0.003687,0.007),
                           'Asz':fish_param(5.5,3,3,nuisance=True),
                            'Aps':fish_param(19.3,3.5,3.5,nuisance=True),
                            'Acib':fish_param(5,2.5,2.5,nuisance=True),
                            'A_TEps' : fish_param(.16,.05,nuisance=True),
                            'Pcal':fish_param(1.06,.01,.01,nuisance=True),
                           'lmax':6000,'lens_potential_accuracy':5.0}

### Planck $\ell$ < 800


In [7]:
pcov_800=create_basic_cov(.7,[.0003],[0,'ukarc'],
                     specrange = (2,800),
                     bin_size = 5,
                     spectra=['TT'],
                     )

full_pcov_800 = make_full_cov(['TT'],pcov_800)

Fp_800_omk,names=Fisher_Matrix(params,full_pcov_800,
                 planck_model(specrange = [('TT',(2,800))],bin_size=5),show_results=False)

pd.DataFrame(Fp_800_omk,index=names,columns=names).to_hdf('Saved_Fisher_Matrices/planck_lcdm_omk_800_Fisher_Matrix.h5','key')

/nfs/home/kmaylor/Forecasting/Covariances/create_basic_cov.py:37: RuntimeWarning: divide by zero encountered in true_divide
  con=lambda n:arcmin_per_sr/n**2


### Planck $\ell$ < 2500 Fisher Matrix

In [21]:
Planck_omk_path="/nfs/home/kmaylor/Official_Planck_chains/base_omegak/plikHM_TTTEEE_lowTEB_lensing/base_omegak_plikHM_TTTEEE_lowTEB_lensing" #loading planck 2015 chain
Planck_omk_chain=K.chains.load_cosmomc_chain(Planck_omk_path).burnin(1000).join()
Planck_omk_chain=add_params(Planck_omk_chain)

cov_params_omk=sorted(['H0','ombh2','ommh2','tau','clamp','ns','omk'])
Planck_omk_cov = Planck_omk_chain.cov(cov_params_omk)
Fp_omk=inv(Planck_omk_cov)

pd.DataFrame(Fp_omk,index=cov_params_omk,columns=cov_params_omk).to_hdf('Saved_Fisher_Matrices/planck_lcdm_omk_Fisher_Matrix.h5','key')

### SPT3G Fisher Matrix

In [ ]:
SPT3G_Fisher_for_lmin(fsky=0.06,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([3.6,3.3,8.5]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_lcdm_omk_lmin_100_700_Fisher_Matrices.h5')

### SPT3G Fisher Matrix fsky =0.035

In [ ]:
SPT3G_Fisher_for_lmin(fsky=0.035,
lmin_range=[100,700],
beam_width=array([0.00032*150/90,.00032,.00032*150/220]),
noise=[array([2.8,2.6,6.6]),'ukarc'],
P_lmax = 5000,
T_lmax = 3000,
spectra = ['TT','EE','TE'],
binn=5,
filename = 'Saved_Fisher_Matrices/SPT3G_f035_lcdm_omk_lmin_100_700_Fisher_Matrices.h5')

###  SPT3G lensing reconstruction Fisher Matrix

In [ ]:
paramsphi={'H0':fish_param(67.90,0.55),#'cosmomc_theta': fish_param(1.04106e-2,0.00041),
                           'ombh2':fish_param(0.02227,0.0002),
                           'ommh2':fish_param(0.1413,0.0011),
                           'tau':fish_param(0.067,0.013, 0.02),
                           'clamp':fish_param(1.873,0.0115),
                           'ns':fish_param( 0.9681,0.0044),
                           'omk':fish_param(-0.003687,0.007),
                           'lmax':6000,'lens_potential_accuracy':5.0}
SPT3G_phi_Fisher_for_lmin(0.06,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_lcdm_omk_lmin_100_700_Fisher_Matrices.h5')

In [ ]:
SPT3G_phi_Fisher_for_lmin(0.035,(100,700),3000,5,'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_omk_lmin_100_700_Fisher_Matrices.h5')

# Load and Combine Planck and SPT FIsher Matrices

### SPT3G 


In [ ]:
Fs_lcdm_lmin_100_700=pd.read_hdf('Saved_Fisher_Matrices/SPT3G_lcdm_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_lcdm_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_lcdm_lmin_100_700_Fisher_Matrices.h5', 'key').add(Fs_lcdm_lmin_100_700)

Fs_lcdm_lmin_50_100=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_lcdm_lmin_50_100_Fisher_Matrices.h5', 'key')

Fs_lcdm_lmin_50_700=pd.concat([Fs_lcdm_lmin_100_700,Fs_lcdm_lmin_50_100.drop('100')])

Fs_f035_lcdm_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_f035_lcdm_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_f035_lcdm_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_lmin_100_700_Fisher_Matrices.h5',
    'key').add(Fs_f035_lcdm_lmin_100_700)

Fs_f035_lcdm_lmin_50_100=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_f035_lcdm_lmin_50_100_Fisher_Matrices.h5', 'key')

Fs_f035_lcdm_lmin_50_700=pd.concat([Fs_f035_lcdm_lmin_100_700,Fs_f035_lcdm_lmin_50_100.drop('100')])

Fs_lcdm_neff_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_lcdm_neff_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_lcdm_neff_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_lcdm_neff_lmin_100_700_Fisher_Matrices.h5',
    'key').add(Fs_lcdm_neff_lmin_100_700)

Fs_lcdm_neff_lmin_50_100=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_lcdm_neff_lmin_50_100_Fisher_Matrices.h5', 'key')

Fs_lcdm_neff_lmin_50_700=pd.concat([Fs_lcdm_neff_lmin_100_700,Fs_lcdm_neff_lmin_50_100.drop('100')])

Fs_f035_lcdm_neff_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_f035_lcdm_neff_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_f035_lcdm_neff_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_lmin_neff_100_700_Fisher_Matrices.h5',
    'key').add(Fs_f035_lcdm_neff_lmin_100_700)

Fs_f035_lcdm_neff_lmin_50_100=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_f035_lcdm_neff_lmin_50_100_Fisher_Matrices.h5', 'key')

Fs_f035_lcdm_neff_lmin_50_700=pd.concat(
    [Fs_f035_lcdm_neff_lmin_100_700,Fs_f035_lcdm_neff_lmin_50_100.drop('100')])

Fs_lcdm_neff_yp_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_lcdm_neff_yp_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_lcdm_neff_yp_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_lcdm_neff_yp_lmin_100_700_Fisher_Matrices.h5',
    'key').add(Fs_lcdm_neff_yp_lmin_100_700)

Fs_lcdm_neff_yp_lmin_50_100=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_lcdm_neff_yp_lmin_50_100_Fisher_Matrices.h5', 'key')

Fs_lcdm_neff_yp_lmin_50_700=pd.concat([Fs_lcdm_neff_yp_lmin_100_700,Fs_lcdm_neff_yp_lmin_50_100.drop('100')])

Fs_f035_lcdm_neff_yp_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_f035_lcdm_neff_yp_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_f035_lcdm_neff_yp_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_neff_yp_lmin_100_700_Fisher_Matrices.h5',
    'key').add(Fs_f035_lcdm_neff_yp_lmin_100_700)

Fs_f035_lcdm_neff_yp_lmin_50_100=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_f035_lcdm_neff_yp_lmin_50_100_Fisher_Matrices.h5', 'key')

Fs_f035_lcdm_neff_yp_lmin_50_700=pd.concat(
    [Fs_f035_lcdm_neff_yp_lmin_100_700,Fs_f035_lcdm_neff_yp_lmin_50_100.drop('100')])

Fs_lcdm_mnu_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_lcdm__mnu_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_lcdm_mnu_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_lcdm_mnu_lmin_100_700_Fisher_Matrices.h5',
    'key').add(Fs_lcdm_mnu_lmin_100_700)

Fs_f035_lcdm_mnu_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_f035_lcdm__mnu_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_f035_lcdm_mnu_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_mnu_lmin_100_700_Fisher_Matrices.h5',
    'key').add(Fs_f035_lcdm_mnu_lmin_100_700)

Fs_lcdm_omk_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_lcdm__omk_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_lcdm_omk_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_lcdm_omk_lmin_100_700_Fisher_Matrices.h5',
    'key').add(Fs_lcdm_mnu_lmin_100_700)

Fs_f035_lcdm_omk_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_f035_lcdm__omk_lmin_100_700_Fisher_Matrices.h5', 'key')

Fs_f035_lcdm_omk_lensing_lmin_100_700=pd.read_hdf(
    'Saved_Fisher_Matrices/SPT3G_phi_f035_lcdm_omk_lmin_100_700_Fisher_Matrices.h5',
    'key').add(Fs_f035_lcdm_mnu_lmin_100_700)